In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
# Define the custom dataset class
class SeriesDataset(Dataset):
    def __init__(self, seq_length, num_samples):
        self.seq_length = seq_length
        self.num_samples = num_samples
        self.data, self.labels = self.generate_series_data(seq_length, num_samples)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    def generate_series_data(self, seq_length, num_samples):
        data = np.zeros((num_samples, seq_length, 1), dtype=np.int32)
        labels = np.zeros((num_samples, 1), dtype=np.int32)

        for i in range(num_samples):
            start = np.random.randint(0, 10)
            data[i, :, 0] = np.arange(start, start + seq_length, dtype=np.int32)
            labels[i, 0] = start + seq_length

        return data, labels

In [3]:
# Create the dataset
seq_length = 10
num_samples = 1000
custom_dataset = SeriesDataset(seq_length, num_samples)

# Split the dataset into train and test sets
train_size = int(0.8 * num_samples)
test_size = num_samples - train_size
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Define the RNN class
class SeriesPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SeriesPredictor, self).__init__()

        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the last output of the sequence
        return out

# Initialize the model, loss function, and optimizer
input_size = 1
hidden_size = 32
output_size = 1
num_layers = 2
model = SeriesPredictor(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs.float(), targets.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/50, Loss: 178.3661315917969
Epoch 2/50, Loss: 136.9672119140625
Epoch 3/50, Loss: 114.7639584350586
Epoch 4/50, Loss: 97.39688079833985
Epoch 5/50, Loss: 82.775361328125
Epoch 6/50, Loss: 70.39114852905273
Epoch 7/50, Loss: 59.80825103759766
Epoch 8/50, Loss: 50.73429321289063
Epoch 9/50, Loss: 43.11390548706055
Epoch 10/50, Loss: 36.60869155883789
Epoch 11/50, Loss: 31.17688446044922
Epoch 12/50, Loss: 26.621536331176756
Epoch 13/50, Loss: 22.89372848510742
Epoch 14/50, Loss: 19.80904281616211
Epoch 15/50, Loss: 17.300405921936036
Epoch 16/50, Loss: 15.28468521118164
Epoch 17/50, Loss: 13.668361778259277
Epoch 18/50, Loss: 12.394663314819336
Epoch 19/50, Loss: 11.395177669525147
Epoch 20/50, Loss: 10.615664081573486
Epoch 21/50, Loss: 10.024611129760743
Epoch 22/50, Loss: 9.574242668151856
Epoch 23/50, Loss: 9.22360008239746
Epoch 24/50, Loss: 8.982188720703125
Epoch 25/50, Loss: 8.789747352600097
Epoch 26/50, Loss: 8.658244075775146
Epoch 27/50, Loss: 8.560068244934081
Epoch 

In [6]:
# Evaluation
model.eval()
with torch.no_grad():
    test_loss = 0
    for inputs, targets in test_loader:
        outputs = model(inputs.float())
        loss = criterion(outputs.float(), targets.float())
        test_loss += loss.item()

        # Print predictions and corresponding expected outputs
        for i in range(len(outputs)):
            print(f"Prediction: {outputs[i][0]:.0f}, Expected: {targets[i][0]:.0f}, Input: {inputs[i].tolist()}")

    print(f"Test Loss: {test_loss / len(test_loader)}")

Prediction: 14, Expected: 18, Input: [[8], [9], [10], [11], [12], [13], [14], [15], [16], [17]]
Prediction: 14, Expected: 11, Input: [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]]
Prediction: 14, Expected: 12, Input: [[2], [3], [4], [5], [6], [7], [8], [9], [10], [11]]
Prediction: 14, Expected: 10, Input: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
Prediction: 14, Expected: 17, Input: [[7], [8], [9], [10], [11], [12], [13], [14], [15], [16]]
Prediction: 14, Expected: 10, Input: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
Prediction: 14, Expected: 18, Input: [[8], [9], [10], [11], [12], [13], [14], [15], [16], [17]]
Prediction: 14, Expected: 14, Input: [[4], [5], [6], [7], [8], [9], [10], [11], [12], [13]]
Prediction: 14, Expected: 19, Input: [[9], [10], [11], [12], [13], [14], [15], [16], [17], [18]]
Prediction: 14, Expected: 18, Input: [[8], [9], [10], [11], [12], [13], [14], [15], [16], [17]]
Prediction: 14, Expected: 18, Input: [[8], [9], [10], [11], [12], [13], [